In [1]:
%AddJar http://dl.bintray.com/spark-packages/maven/cloudant-labs/spark-cloudant/2.0.0-s_2.11/spark-cloudant-2.0.0-s_2.11.jar

Using cached version of spark-cloudant-2.0.0-s_2.11.jar


In [2]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.storage.StorageLevel
import org.apache.spark.streaming.{Time, Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}

val conf = sc.getConf
conf.set("cloudant.host","83a4d5b3-b0cd-4206-a1dd-b8ede559ef88-bluemix.cloudant.com")
conf.set("cloudant.username", "83a4d5b3-b0cd-4206-a1dd-b8ede559ef88-bluemix")
conf.set("cloudant.password","004f4adfb5a46442c3a5e37f6d62eec477c9f3d96781d45f612bfba6afd4b013")

conf.setJars(ClassLoader.getSystemClassLoader.asInstanceOf[java.net.URLClassLoader].getURLs.map(_.toString).toSet.toSeq ++ kernel.interpreter.classLoader.asInstanceOf[java.net.URLClassLoader].getURLs.map(_.toString).toSeq)
conf.set("spark.driver.allowMultipleContexts", "true")
conf.set("spark.master","local[*]")
val scCloudant = new SparkContext(conf)
scCloudant.getConf.getAll.foreach(println)


(cloudant.password,004f4adfb5a46442c3a5e37f6d62eec477c9f3d96781d45f612bfba6afd4b013)
(spark.eventLog.enabled,true)
(spark.deploy.resourceScheduler.factory,org.apache.spark.deploy.master.EGOResourceSchedulerFactory)
(cloudant.host,83a4d5b3-b0cd-4206-a1dd-b8ede559ef88-bluemix.cloudant.com)
(spark.r.command,/usr/local/src/bluemix_jupyter_bundle.v31/R/bin/Rscript)
(spark.jars,file:/usr/local/src/dataconnector-dw/spark-1.6.0/Server/connectivity/thirdparty/JSON4J-1.1.0.0.jar,file:/usr/local/src/dataconnector-dw/spark-1.6.0/Server/connectivity/thirdparty/commons-io-2.4.jar,file:/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb8e-31a08100070616-ac6107fb81b2/data/libs/scala-2.10/*,file:/usr/local/src/dataconnector-dw/spark-1.6.0/Server/connectivity/thirdparty/httpmime.jar,file:/usr/local/src/dataconnector-stocator-1.6/spark-1.6.0/libs/stocator-1.0.7.jar,file:/usr/local/src/analytic-libs/spark-1.6.0/ia-stan-classifiers-201611220127.jar,file:/usr/local/src/dataconnector-dw/spark-1.6.0/Server/conne

In [3]:
def readFromDatabase(sqlContext: SQLContext, databaseName: String) = {
    val df = sqlContext.read.format("com.cloudant.spark").load(databaseName)
    df
}

In [4]:
val sqlContext = new SQLContext(scCloudant)
import sqlContext.implicits._
val df = readFromDatabase(sqlContext, "meetup_group")
df.show(5)
df.filter(df("group_city")==="Austin").show(5)


Use connectorVersion=1.6.3, dbName=meetup_group, indexName=null, viewName=null,jsonstore.rdd.partitions=5, + jsonstore.rdd.maxInPartition=-1,jsonstore.rdd.minInPartition=10, jsonstore.rdd.requestTimeout=900000,bulkSize=20, schemaSampleSize=1
+--------------------+--------------------+------------+-------------+--------+---------+---------+--------------------+-----------+--------------------+--------------------+
|                 _id|                _rev|  group_city|group_country|group_id|group_lat|group_lon|          group_name|group_state|        group_topics|       group_urlname|
+--------------------+--------------------+------------+-------------+--------+---------+---------+--------------------+-----------+--------------------+--------------------+
|270248487114c389b...|1-80c343f3d28e074...| San Antonio|           us|  931700|     29.5|    -98.4|The San Antonio N...|         TX|[[Dogs,dogs], [Do...|  sanaturehounds-com|
|2dfaa27c519063906...|1-3f2e6c01623b457...|      Dallas|  